# Project 01 - Color Compression

## Thông tin sinh viên

- Họ và tên: **Nguyễn Công Tuấn**
- MSSV: **22127436**
- Lớp: **22CLC03**

## Import các thư viện liên quan

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

## Helper functions

In [ ]:
def read_img(img_path):
    image = Image.open(img_path)
    return np.array(image)

def show_img(img_2d):
    plt.imshow(img_2d)
    plt.axis('off')
    plt.show()

def save_img(img_2d, img_path):
    image = Image.fromarray(img_2d)
    image.save(img_path)

def convert_img_to_1d(img_2d):
    return img_2d.reshape((-1, img_2d.shape[2]))

def kmeans(img_1d, k_clusters, max_iter=100, init_centroids='random'):
    centroids = initialize_centroids(img_1d, k_clusters, init_centroids)

    for _ in range(max_iter):
        labels = assign_labels(img_1d, centroids)
        new_centroids = update_centroids(img_1d, centroids, labels, k_clusters)

        if np.all(centroids == new_centroids):
            break

        centroids = new_centroids

    return centroids, labels

def generate_2d_img(img_2d_shape, centroids, labels):
    new_img_1d = centroids[labels]
    return new_img_1d.reshape(img_2d_shape).astype(np.uint8)

def initialize_centroids(img_1d, k_clusters, init_method):
    if init_method == 'random':
        return np.random.randint(0, 256, size=(k_clusters, img_1d.shape[1]))
    elif init_method == 'in_pixels':
        indices = np.random.choice(img_1d.shape[0], k_clusters, replace=False)
        return img_1d[indices]

def assign_labels(img_1d, centroids):
    distances = np.linalg.norm(img_1d[:, np.newaxis] - centroids, axis=2)
    return np.argmin(distances, axis=1)

def update_centroids(img_1d, centroids, labels, k_clusters):
    new_centroids = []
    for i in range(k_clusters):
        points_in_cluster = img_1d[labels == i]
        if len(points_in_cluster) == 0:
            new_centroids.append(np.random.randint(0, 256, size=(img_1d.shape[1])))
        else:
            new_centroids.append(points_in_cluster.mean(axis=0))
    return np.array(new_centroids)

## Your tests

## Main FUNCTION

In [ ]:
def main():
    img_path = input("Enter the image path: ")
    img_2d = read_img(img_path)

    print("Original Image")
    show_img(img_2d)

    img_1d = convert_img_to_1d(img_2d)
    k = int(input("Enter the number of colors for K-Means: "))
    max_iter = int(input("Enter the maximum number of iterations for K-Means: "))
    init_method = input("Enter the initialization method (random/in_pixels): ").strip().lower()

    while init_method not in ['random', 'in_pixels']:
        print("Invalid initialization method. Please enter 'random' or 'in_pixels'.")
        init_method = input("Enter the initialization method (random/in_pixels): ").strip().lower()

    centroids, labels = kmeans(img_1d, k_clusters=k, max_iter=max_iter, init_centroids=init_method)
    img_2d_output = generate_2d_img(img_2d.shape, centroids, labels)

    print(f"Image after K-means algorithm with k = {k}")
    show_img(img_2d_output)

    save_image_path = []
    save_image_path.append(img_path.rsplit('.', 1)[0] + '_result.png')
    save_image_path.append(img_path.rsplit('.', 1)[0] + '_result.pdf')

    for path in save_image_path:
        save_img(img_2d_output, path)

    print('Save Successfully')

In [ ]:

main()